## Scraping

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [2]:
def get_tracks(page_url):
    
    # Setup soup for main page
    page_r = requests.get(page_url)
    page_soup = BeautifulSoup(page_r.text, "html.parser")
    page_html = page_soup.body.div.find_all("a")
    
    # Setup empty list and fill it with the URLS
    track_list = []
    for i in range(33,84, 5): # these are the positions where we find the track urls in this case
        entry = page_html[i]["href"]
        track_list.append(entry)
        
    return track_list

In [3]:
get_tracks("https://www.bensound.com/royalty-free-music/")

['https://www.bensound.com/royalty-free-music/track/creative-minds',
 'https://www.bensound.com/royalty-free-music/track/a-new-beginning',
 'https://www.bensound.com/royalty-free-music/track/little-idea',
 'https://www.bensound.com/royalty-free-music/track/jazzy-frenchy',
 'https://www.bensound.com/royalty-free-music/track/happy-rock',
 'https://www.bensound.com/royalty-free-music/track/hey-happy-cheerful',
 'https://www.bensound.com/royalty-free-music/track/cute',
 'https://www.bensound.com/royalty-free-music/track/memories',
 'https://www.bensound.com/royalty-free-music/track/going-higher',
 'https://www.bensound.com/royalty-free-music/track/acoustic-breeze',
 'https://www.bensound.com/royalty-free-music/track/buddy']

In [4]:
def get_tags(track_url):
    
    # Setup soup for track page
    track_r = requests.get(track_url)
    track_soup = BeautifulSoup(track_r.text, "html.parser")
    taglist_html = track_soup.body.div.find("div", {"class": "taglist"})
    taglist_html = taglist_html.find_all("a")
    taglist = []
    for i in taglist_html:
        for j in i:
            taglist.append(j)          
    return taglist

In [5]:
def get_tags(track_url):
    
    # Setup soup for track page
    track_r = requests.get(track_url)
    track_soup = BeautifulSoup(track_r.text, "html.parser")
    taglist_html = track_soup.body.div.find("div", {"class": "taglist"})
    taglist_html = taglist_html.find_all("a")
    # schauen ob download for free
    taglist = []
    try:
        license = track_soup.body.div.find("div", {"id": "freelicense"})
        license = license.find_all("h3")
        for i in taglist_html:
            for j in i:
                taglist.append(j)
    except:
        print('Song not for free')
    
    return taglist

In [6]:
get_tags("https://www.bensound.com/royalty-free-music/track/all-the-way-up-uplifting-pop-rock-awesome")

Song not for free


[]

In [7]:
get_tags("https://www.bensound.com/royalty-free-music/track/ukulele")

['ukulele',
 'happy',
 'funny',
 'advertising',
 'upbeat',
 'kid',
 'kids',
 'positive',
 'chidren',
 'joy',
 'fun',
 'acoustic',
 'light',
 'gentle']

In [8]:
url_template = "https://www.bensound.com/royalty-free-music/{page}"
page_urls = ["https://www.bensound.com/royalty-free-music/"]

In [9]:
for i in range(2,24):
    page_urls.append(url_template.format(page = i))

In [10]:
def get_all_tags(page_urls):
    
    # Setup the dict that we will create the df from later
    tag_dict = {"URL" : [], "Tags" : []}
    
    # Loop through all page urls and extract the track urls
    for i, page_url in enumerate(page_urls):    
        track_list = get_tracks(page_url)
        
        # Loop through the track urls and extract the revleant information
        for track_url in track_list:
            tag_list = get_tags(track_url)
            tag_dict["URL"].append(track_url)
            tag_dict["Tags"].append(tag_list)
    
    return tag_dict

In [11]:
tag_dict = get_all_tags(page_urls)

Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not for free
Song not f

In [12]:
df_tags = pd.DataFrame(tag_dict)
df_tags.head()
df_tags.shape

# Export csv
#df_tags.to_csv("csv/music_tags_raw.csv", sep = ";", index = False)

(253, 2)

## Data preprocessing

In [13]:
# import csv
music_tags = pd.read_csv("csv/music_tags_raw(1).csv", sep = ";")
music_tags.shape

(76, 2)

In [14]:
# create new column "track_name"
# the track names start at index 50 in every url
music_tags["track_name"] = music_tags["URL"].str.slice(start = 50)

# replace "-" with space
music_tags["track_name"] = music_tags["track_name"].str.replace("-", " ")

#music_tags["track_name"].head()

# Change type of Tag list from string to list object
# create empty list
tag_lists = []

# loop through
for string in music_tags["Tags"]:
    string = eval(string)
    tag_lists.append(string)
    
music_tags["Tags"] = tag_lists

music_tags.head()

,URL,Tags,track_name
0,https://www.bensound.com/royalty-free-music/tr...,"[nature, simple, light, happy, dreamy, dream, ...",acoustic breeze
1,https://www.bensound.com/royalty-free-music/tr...,"[rock, action, aggressive, violent, tough, str...",actionable
2,https://www.bensound.com/royalty-free-music/tr...,"[acoustic, gentle, mellow, sweet, calm, guitar...",a day to remember wedding music
3,https://www.bensound.com/royalty-free-music/tr...,"[travel, nature, discovery, film, adventure, m...",adventure
4,https://www.bensound.com/royalty-free-music/tr...,"[hip, hop, chill, jazz, lounge, samples, groov...",all that chill hop


In [15]:
# save all tags in dict
all_tags = {}

# Loop through every list
for tag_list in music_tags["Tags"]:
    
    # Loop through every element in the list
    for tag in tag_list:
        # Add them to the dict or increment their frequency by 1
        if tag not in all_tags:
            all_tags[tag] = 1
        else:
            all_tags[tag] += 1

all_tags_sorted = {k: v for k, v in sorted(all_tags.items(), key=lambda item: item[1])}
#all_tags_sorted

In [16]:
def get_all_unique(dataframe_of_lists):
    # Create empty list
    unique_tags = []
    
    # Loop through the dataframe rows (lists) and each item inside
    for row in dataframe_of_lists:
        for item in row:
            
            # Add item to all_tags if it's not already in it
            if item not in unique_tags:
                unique_tags.append(item)
                
    return unique_tags

In [17]:
def create_boolean_df(unique_tags, track_taglists):
    # Create new df with a column for every tag
    boolean_df = pd.DataFrame(columns = unique_tags)
    boolean_df.shape
    
    # Create an empty dict
    data_dict = {}
    
    # Loop through the columns (tags) in the boolean_df and add them to the dict
    for col in boolean_df:
        data_dict[col] = []
        
        # Loop through the taglists in the old dataframe
        for taglist in track_taglists:
            
            # Check if the column (tag) is in the tracks taglist. If yes append True else append False
            data_dict[col].append(col in taglist)
    
    # Use the boolean lists as values for the boolean_df's columns
    for col in boolean_df:
        boolean_df[col] = data_dict[col]
        
    return boolean_df

In [18]:
unique_tags = get_all_unique(music_tags["Tags"])
music_tags_bool = create_boolean_df(unique_tags, music_tags["Tags"])
print(music_tags_bool.shape)

music_tags_bool = music_tags_bool.set_index(music_tags["track_name"])
music_tags_bool.head()

#export
#music_tags_bool.to_csv("csv/music_tags_bool.csv")

(76, 721)


,nature,simple,light,happy,dreamy,dream,optimistic,inspirational,hopeful,positive,cheerful,bright,sun,summer,warm,sweet,soft,romantic,slideshow,photo,album,wedding,sentimental,peaceful,gentle,cello,guitar,folk,acoustic,rock,action,aggressive,violent,tough,strong,sport,extreme,rough,power,powerful,overdrive,intense,hard,electric,distortion,saturated,angry,violence,mellow,calm,love,photos,soulful,strings,piano,emotional,elegant,moving,video,touching,couple,travel,discovery,film,adventure,movie,journey,orchestral,orchestra,documentary,cinematic,inspiring,timpani,marimba,hip,hop,chill,jazz,lounge,samples,groove,groovy,urban,sax,saxophone,old,laid back,beat,modern,street,background,youtube,royalty-free,chillout,chill out,catchy,advertising,commercial,uplifting,success,hope,motivation,invincible,corporate,epic,energy,energetic,confidence,business,funky,suspense,cop,cops,retro,opening,bongoes,ambient,electric guitar,landscape,ocean,dramatic,spiritual,meditation,sad,slow,soundtrack,pad,melancholic,emotion,inspiration,presentation,pop,indie,hollywood,battle,war,choir,intensity,trailer,courage,hero,heroic,game,brazil,brazilian,samba,party,carnival,upbeat,vacations,rio,parade,joy,dance,beach,cute,fun,kids,children's,fresh,claps,clapping,glockenspiel,bells,pian,ukulele,ad,funny,cartoon,bouncy,animation,morning,drums,sunshine,impulse,wake,up,alarm,clarity,boost,stamina,clear,sky,productive,breakfast,playful,playtime,sunny,vitality,country,bluegrass,banjo,cowboy,cowboys,farmer,farm,redneck,rodeo,violin,amaerica,usa,south,motivational,palm mute,serious,innovate,creative,innovation,science,technology,creepy,scary,weird,worrying,circus,freak,show,dark,horror,halloween,celesta,accordion,waltz,music,box,children,child,baby,babies,nice,house,electronica,synth,synthpad,techno,rhythm,club,drinks,night,disco,hallucination,city life,delay,disarray,speed,fast,snowboard,mountain,advert,fashion,catwalk,model,photoshoot,magazine,alcohol,product,drink,dancing,move,drugs,pill,sordid,craziness,electronic,confusion,bike,teens,rampage,race,car,rallycross,racing,motocross,motorbike,skate,parkour,youth,F1,danger,adrenaline,competition,challenge,keyboard,galaxy,seabed,abyss,stars,space,opera,exploration,silence,peace,sea,diving,water,relaxation,underwater,blue,deep,funk,brass,horns,trumpet,blues,congas,rhodes,drum solo,relax,relaxing,future,tropical,edm,electro,vocals,vlog,pluck,beautiful,chic,dubstep,cars,wobble,hi,tech,agressive,bass,synthetic,pulsing,analog,pumping,vocal chops,synthesizer,dj,vocal,professional,exotic,shaker,africa,african,enigmatic,mysterious,anxious,odd,magic,magician,kalimba,mistery,triumphant,impact,fight,score,cinema,french,glitch,daft punk,driving,loud,trendy,woodwinds,video game,motor,free,ride,metal,corporate music,vintage,organ,joyful,percussion,promotional,tutorial,education,educational,outdoors,city,vibes,pursuit,tv,countdown,cool,run,contrabass,shaft,starsky,hutch,eleven,tension,sneaky,smooth,robbery,seventies,70,wah,upright,stealth,xylophone,joke,amusing,childish,humorous,lively,cheery,beats,scratching,victory,achievenment,triumph,climax,rap,scratch,bad,boy,dancer,breakdance,break,breakbeat,alternative,wellness,vacation,leisure,happ,happiness,winner,winning,distorded,hand,kid,hey,smile,week,project,top,luxury,double,beauty,sexy,shooting,designer,crowd,French,cosmetics,nightlife,citylife,runway,mannequin,fashionable,haute,couture,high,outfit,style,clothing,VIP,artist,elegance,trend,glam,temptation,shop,appeal,looks,girl,jewel,perfume,fragrance,makeup,cream,luxurious,rich,paris,expensive,hedonism,pleasure,lavish,skincare,hairstyle,haircut,hairdo,india,indian,spirituality,asia,asian,world,sitar,tablas,zen,hindu,yoga,vibraphone,comedy,humor,quartet,burlesque,entertainment,goofy,farce,prank,silly,laidback,easy,listening,absurd,naive,simplicity,comedian,instrumental,jazzy,stupid,bar,laughter,cooking,brushes,family,swing,django,france,gypsy,delight,ethnic,arab,wild,oriental,new,age,morocco,maghreb,soundcape,ecology,ballad,serenity,cockta

## Reduce tags

In [19]:
# import
data = pd.read_csv("csv/music_tags_bool.csv", sep= ",", index_col= 0)
data.shape

(76, 721)

In [20]:
for i in range(0,721):
    tag_name = data.columns[i]
    #print(tag_name)
    count = data.loc[:,tag_name].value_counts()[1]
    #print(count)
    data.loc['count_true',tag_name] = count

for i in range(0,76):
    song = data.index[i]
    data.loc[song,'count_trues'] = 0

In [21]:
data = data.drop(['advertising'], axis = 1)

for i in data:
    #tag_name = data.columns[i]
    if data.loc['count_true', i] <= 15:
        data = data.drop([i], axis = 1)

for i in range(0,76):
    count1 = data.iloc[i,:].value_counts()
    #print(count1[False])
    col = data.shape[1]
    #print(col)
    song = data.index[i]
    #print(song)
    data.loc[song,'count_trues'] = col - count1[False]

data

,happy,positive,guitar,piano,commercial,upbeat,fun,count_trues
track_name,,,,,,,,
acoustic breeze,True,True,True,False,False,False,False,3.0
actionable,False,False,True,False,False,False,False,1.0
a day to remember wedding music,False,False,True,True,False,False,False,2.0
adventure,False,False,False,False,False,False,False,0.0
all that chill hop,False,True,False,True,True,False,False,3.0
a new beginning,False,True,True,True,False,False,False,3.0
badass,False,False,True,False,False,False,False,1.0
better days,False,False,False,False,False,False,False,0.0
beyond the line,False,True,True,False,False,False,False,2.0


In [22]:
# songs we will loose
a = data.shape[0]
for i in range(0,a):
    #tag_name = data.columns[i]
    song = data.index[i]
    if data.loc[song, 'count_trues'] <= 0:
        print(song)

adventure
better days
birth of a hero
creepy
deep blue
dubstep
enigmatic
epic
evolution epic nature
groovy hip hop
india
little planet
new dawn
ofelias dream
psychedelic
relaxing
sci fi
sexy
straight
the duel
the elevator bossa nova
the lounge


In [23]:
final_tag_list = data[data['count_trues'] > 0]

# drop counting row and column
final_tag_list = final_tag_list.drop(['count_trues'], axis = 1)
final_tag_list

#export
#final_tag_list.to_csv("csv/music_tags_reduced.csv")

,happy,positive,guitar,piano,commercial,upbeat,fun
track_name,,,,,,,
acoustic breeze,True,True,True,False,False,False,False
actionable,False,False,True,False,False,False,False
a day to remember wedding music,False,False,True,True,False,False,False
all that chill hop,False,True,False,True,True,False,False
a new beginning,False,True,True,True,False,False,False
badass,False,False,True,False,False,False,False
beyond the line,False,True,True,False,False,False,False
brazil samba,True,False,False,False,False,True,False
buddy,True,True,False,False,True,False,True
